In [2]:
#Write to markdown file
def write_to_markdown(file_name='sample.md', input=""):
    with open(file_name, 'w') as file:
        file.write(input)

In [3]:
source_pdf = "/Users/sambitmahapatra/Downloads/Merged Cell Table.pdf"

In [4]:
from docling.document_converter import DocumentConverter

# Create a DocumentConverter instance
converter = DocumentConverter()

# Convert the PDF to Markdown
result = converter.convert(source_pdf)

# Export the result to Markdown format
markdown_output = result.document.export_to_markdown()
write_to_markdown(file_name='sample_docling.md', input=markdown_output)

/Users/sambitmahapatra/Documents/hobglobin_code_folder/haystack-app/devenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 70295.60it/s]


In [5]:
import pymupdf4llm

# Convert the PDF to Markdown format
markdown_output = pymupdf4llm.to_markdown(source_pdf)

write_to_markdown(file_name='sample_pymupdf4llm.md', input=markdown_output)

Processing /Users/sambitmahapatra/Downloads/Merged Cell Table.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]


In [6]:
#Reference: https://dev.to/rishabdugar/pdf-extraction-retrieving-text-and-tables-together-using-python-14c2

import pdfplumber
import pandas as pd
from pdfplumber.utils import extract_text, get_bbox_overlap, obj_to_bbox
def convert_pdf_to_markdown(pdf_path):
    pdf = pdfplumber.open(pdf_path)
    all_text = []
    for page in pdf.pages:
        filtered_page = page
        chars = filtered_page.chars
        for table in page.find_tables():
            first_table_char = page.crop(table.bbox).chars[0]
            filtered_page = filtered_page.filter(lambda obj: 
                get_bbox_overlap(obj_to_bbox(obj), table.bbox) is None
            )
            chars = filtered_page.chars
            df = pd.DataFrame(table.extract())
            df.columns = df.iloc[0]
            markdown = df.drop(0).to_markdown(index=False)
            chars.append(first_table_char | {"text": markdown})
        page_text = extract_text(chars, layout=True)
        all_text.append(page_text)
    pdf.close()
    return "\n".join(all_text)

markdown_output = convert_pdf_to_markdown(source_pdf)

write_to_markdown(file_name='sample_pdfplumber.md', input=markdown_output)